
# Simple generator

- 1 frame : 1 second




In [321]:
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from skimage.draw import random_shapes, rectangle
from skimage.util import img_as_ubyte
import pandas as pd
import datetime, dateutil

In [322]:
%matplotlib inline

In [323]:
# which shapes to 'choose' from
shape_arr = ['triangle', 'circle']
frame_size = (1080,1920)

# which shape is the conditioned stimulus
other_stim = 'circle'
cond_stim = 'triangle'

# randomly selects shapes (must be even!)
number_of_shapes = 20

## Create fixation stimulus

In [324]:
h, w = frame_size
blank_img = np.full((h, w), 255, dtype = np.uint8)

In [325]:
fix_img = np.full((h, w), 255, dtype = np.uint8)
start = (540,960)
extent = (590,1010)
rr, cc = rectangle(start, end=extent, shape=fix_img.shape)
fix_img[rr, cc] = 0

## Create image arrays

In [326]:
# each holds camera images 
cam1_images = []
cam2_images = []
cam3_images = []

# order dfs for writing time keys
time_order = []
shape_order = []

In [327]:
def cam_appender(img_folder, img, n_repeats):
    for _ in range(n_repeats):
        img_folder.append(img)

def cam3_fixation():
    ''' Fixation pt for 10 seconds
        followed by 15 second blank frame.
    '''
    cam_appender(cam3_images,fix_img, 10)
    append_single(cam3_images, 15)

def append_single(cam, num):
    ''' for appending blank images to a single camera.
            cam = name of camera image folder;
            num = number of blank frames to append
    '''
    for _ in range(num):
        cam.append(blank_img)

def append_multiple(first_cam, second_cam, num):
    ''' for appending blank images to a single camera.
            first_cam = name of camera image folder;
            second_cam = name of other camera image folder;
            num = number of blank frames to append
    '''
    for _ in range(num):
        first_cam.append(blank_img)
        second_cam.append(blank_img)

def get_results(shapes, camera_images, num):
    result = random_shapes((140,140), intensity_range=(0, 1), max_shapes=1, min_size = 99, max_size=100, shape=shapes, multichannel=False)
    result_pad = np.pad(result[0], ((470,470),(890,890)), mode='constant', constant_values=255)
    cam_appender(camera_images,result_pad, num)
    append_single(camera_images, int(5))


In [328]:
shapes = np.random.choice(shape_arr, number_of_shapes, p=[0.5,0.5])

## Blank into followed by fixation

1. 1 minute (60 frames) blank
2. 25 second fixation

In [329]:
# Generates first frame
# video 3 will start with a 1 second blank
# video 1 and 2 will begin with 6 second blank
append_single(cam3_images, int(60))

cam3_fixation()

append_multiple(cam1_images, cam2_images, int(60+25))
n_secs = 10

In [330]:
for cam1_shapes, cam2_shapes in zip(shapes, shapes[::-1]):
    # randomly chooses between camera 1 and 2 
    cam_array = ['cam1', 'cam2']
    which_array = np.random.choice(cam_array, 1, p=[0.5,0.5])

    # if camera 1 is chosen
    if which_array == 'cam1':

        shape_order.append([cam1_shapes, 'fixation', cam2_shapes, 'fixation'])
        time_order.append(['camera 1', 'fixation', 'camera 2', 'fixation'])
        get_results(cam1_shapes,cam1_images, int(n_secs))
        
        append_multiple(cam2_images, cam3_images, int(n_secs + 5))

        # camera 3 fixation
        
        cam3_fixation()

        # for camera 1 and 2 (delay from fixation point)
        append_multiple(cam1_images, cam2_images, int(25))

        get_results(cam2_shapes,cam2_images, int(n_secs))
        append_multiple(cam1_images, cam3_images, int(n_secs + 5))

        # camera 3 only
        cam3_fixation()

        append_multiple(cam1_images, cam2_images, int(25))

    # if camera 2 is chosen,
    else:
        shape_order.append([cam2_shapes, 'fixation', cam1_shapes, 'fixation'])
        time_order.append(['camera 2', 'fixation', 'camera 1', 'fixation'])

        get_results(cam2_shapes,cam2_images, int(n_secs))

        append_multiple(cam1_images, cam3_images, int(n_secs + 5))

        # camera 3 only
        cam3_fixation()

        # for camera 1 and 2 (delay from fixation point)
        append_multiple(cam1_images, cam2_images, int(25))

        # append camera 2
        get_results(cam1_shapes,cam1_images, int(n_secs))
        append_multiple(cam2_images, cam3_images, int(n_secs + 5))

        # camera 3 only
        cam3_fixation()
        append_multiple(cam1_images, cam2_images, int(25))


## Generate Time Key

Generate a key to videos to know when particular stimuli occur

In [331]:
time_order = np.array(time_order).flatten()
time_order = np.insert(time_order,0,'fixation')
shape_order =  np.array(shape_order).flatten()
shape_order = np.insert(shape_order,0,'fixation')
# generating timekey 

time_key = pd.DataFrame({'camera_order':time_order, 'shape_order':shape_order})
time_key['duration'] = np.nan
time_key.loc[time_key['shape_order']=='fixation', 'duration']= 25
time_key.loc[time_key['shape_order']!='fixation', 'duration']= 15

# shift since first frame begins with blank frame (1sec) 
time_key.at[0,'duration'] = time_key.at[0,'duration'] + 60
time_key['times'] = time_key['duration'].cumsum()
time_key['times'] = pd.to_timedelta(time_key['times'], unit='s')
time_key['times'] = time_key['times'].shift(1)
time_key.at[0,'times'] = 60

# Save times
today = datetime.datetime.now()
today = today.strftime("%m-%d-%Y-%H-%M-%S")
time_key_df_name = '{}_{}'.format(today, 'time_key.csv')
time_key.to_csv(time_key_df_name)

## Writing videos to folders

In [332]:
# writing videos
fourcc = cv2.VideoWriter_fourcc(*'avc1')
fps=1

In [333]:
video_name = '{}{}{}'.format('cam_01_', today, '.mp4' )
video = cv2.VideoWriter(video_name, fourcc, fps , (w,h), 0)

for frame in cam1_images:
    frame = img_as_ubyte(frame)
    video.write(frame)
video.release()

In [334]:
video_name = '{}{}{}'.format('cam_02_', today, '.mp4' )
video = cv2.VideoWriter(video_name, fourcc, fps , (w,h), 0)

for frame in cam2_images:
    frame = img_as_ubyte(frame)
    video.write(frame)
video.release()

In [335]:
video_name = '{}{}{}'.format('cam_03_', today, '.mp4' )
video = cv2.VideoWriter(video_name, fourcc, fps , (w,h), 0)

for frame in cam3_images:
    frame = img_as_ubyte(frame)
    video.write(frame)
video.release()